In [1]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# data preprocessing
kdd_data = open('/content/KDDTrain+.txt','r')
test_handle = open('/content/test.txt','a')
for i in kdd_data.readlines():
  last_comma=i.rfind(',')
  sec_last_comma = i.rfind(',',last_comma)
  label = i[sec_last_comma+1:last_comma]
  test_handle.write(label)
  test_handle.write('/n')
  i = i[:sec_last_comma]+i[last_comma:]



kdd_data.close()
test_handle.close()

kdd_data = open('/content/KDDTest-21.txt','r')
test_handle = open('/content/test_20.txt','a')
for i in kdd_data.readlines():
  last_comma=i.rfind(',')
  sec_last_comma = i.rfind(',',last_comma)
  label = i[sec_last_comma+1:last_comma]
  test_handle.write(label)
  test_handle.write('/n')
  i = i[:sec_last_comma]+i[last_comma:]
train = open('/content/KDDTrain+.txt','r')
test = open('/content/test.txt','r')
train_20 = open('/content/KDDTest-21.txt','r')
test_20 = open('/content/test_20.txt','r')
# load data
x_train,x_test,y_train,y_test = train,test,train_20,test_20
# Load the VGG16 model with pre-trained weights, excluding the top layers
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_height, image_width, 3))

# # Freeze the base model's layers
# for layer in base_model.layers:
#     layer.trainable = False

# Add LSTM layers to the modele
model = Sequential()
# model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
# model.add(Reshape((1, -1)))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(32))
model.add(Dense(num_classes=5, activation='softmax')) # nmap  , normal , ipsweep, neptune , teardown

# Compile the model with a lower learning rate and customized optimizer
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Implement callbacks for better training control
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.00001)

# Train the model with augmented data if available
history = model.fit(x_train, y_train, batch_size=32, epochs=20, callbacks=[early_stopping, reduce_lr])

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

# Plotting the training history for analysis
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()